In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.optimizers import Adam

2023-11-18 12:16:07.909522: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-18 12:16:08.135887: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-18 12:16:08.137495: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-18 12:16:09.471956: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# 1. Load the dataset (MNIST)
(x_train, _), (x_test, _) = mnist.load_data()  # We ignore labels for unsupervised pretraining

In [3]:
# 2. Preprocess the dataset
# Normalize and flatten the images
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [4]:
# 3. Define the unsupervised learning model (AutoEncoder)
# We will create a simple autoencoder for demonstration purposes
def create_autoencoder(input_dim, encoding_dim):
    autoencoder = Sequential()
    autoencoder.add(Dense(encoding_dim, input_shape=(input_dim,), activation='relu'))
    autoencoder.add(Dense(input_dim, activation='sigmoid'))
    autoencoder.compile(optimizer=Adam(), loss='binary_crossentropy')
    return autoencoder

In [7]:
# Function to create encoder model
def create_encoder(input_dim, encoding_dim):
    encoder = Sequential()
    encoder.add(Dense(encoding_dim, input_shape=(input_dim,), activation='relu'))
    return encoder

In [9]:
# 4. Pretrain each layer of the model with the dataset
encoding_dims = [512, 256, 128]
input_dim = x_train.shape[1]
for encoding_dim in encoding_dims:
    # Create an autoencoder for the current layer
    autoencoder = create_autoencoder(input_dim, encoding_dim)
    # Pretrain the autoencoder
    autoencoder.fit(x_train, x_train,
                    epochs=50,
                    batch_size=256,
                    shuffle=True,
                    validation_data=(x_test, x_test),
                    verbose=0)
    
    # Create a corresponding encoder model for encoding
    encoder = create_encoder(input_dim, encoding_dim)
    # Set weights to the encoder part of the autoencoder
    encoder.set_weights([autoencoder.layers[0].get_weights()[0], autoencoder.layers[0].get_weights()[1]])
    
    # Use the encoder to generate the input for the next layer
    x_train = encoder.predict(x_train)
    x_test = encoder.predict(x_test)
    
    # Update input_dim for the next layer
    input_dim = encoding_dim

print("Greedy layer-wise pretraining completed.")

313/313 [==============================] - 0s 1ms/step
Greedy layer-wise pretraining completed.


In [10]:
"""
1875/1875 [==============================] - 3s 1ms/step
313/313 [==============================] - 0s 1ms/step
1875/1875 [==============================] - 2s 1ms/step
313/313 [==============================] - 0s 1ms/step
1875/1875 [==============================] - 2s 1ms/step
313/313 [==============================] - 0s 1ms/step
Greedy layer-wise pretraining completed.
"""

'\n1875/1875 [==============================] - 3s 1ms/step\n313/313 [==============================] - 0s 1ms/step\n1875/1875 [==============================] - 2s 1ms/step\n313/313 [==============================] - 0s 1ms/step\n1875/1875 [==============================] - 2s 1ms/step\n313/313 [==============================] - 0s 1ms/step\nGreedy layer-wise pretraining completed.\n'